In [2]:
# ETF_Cache.ipynb

import time
import pandas as pd
from ETF_Tickers import ETF_Ticker_List
from stock_cache import (
    fetch_stock_data_bulk,
    stock_df_from_cache,
    method_df_from_cache,
    save_cache_parquet,
    load_cache_parquet_if_fresh
)
from main_table import build_quick_table

# -----------------------------
# CONFIG
# -----------------------------
ETF_PATH = r"iShares-Core-MSCI-World-UCITS-ETF_fund.csv"
CACHE_PATH = "etf_cache.parquet"
TOP_N = None          # None = all tickers
MAX_AGE_HOURS = 24    # freshness for "quick load"
MAX_WORKERS = 4       # lower concurrency to reduce rate limits
BATCH = 100           # fetch in chunks; 150–250 is reasonable
PAUSE_SEC = 12         # pause between batches to be polite
# -----------------------------


def refresh_cache(tickers, path=CACHE_PATH):
    """
    Load existing cache (even if old), fetch missing tickers,
    save merged cache, and return it.
    """
    existing_cache = load_cache_parquet_if_fresh(path, max_age_hours=1e9) or {}
    missing = [t for t in tickers if t not in existing_cache]

    if not missing:
        print("✅ No missing tickers — cache is complete.")
        return existing_cache

    print(f"Need to fetch {len(missing)} tickers.")
    for i in range(0, len(missing), BATCH):
        batch = missing[i:i+BATCH]
        print(f"Fetching batch {i//BATCH + 1} / {((len(missing)-1)//BATCH)+1} — {len(batch)} tickers...")
        new_data = fetch_stock_data_bulk(batch, max_workers=MAX_WORKERS)
        # keep successful only
        existing_cache.update({k: v for k, v in new_data.items() if v})
        save_cache_parquet(existing_cache, path)
        print(f"Saved cache after {i+len(batch)} tickers.")
        time.sleep(PAUSE_SEC)

    return existing_cache


# -----------------------------
# MAIN EXECUTION
# -----------------------------
tickers = ETF_Ticker_List(ETF_PATH, top_n=TOP_N)
print(f"Loaded {len(tickers)} tickers from ETF list.")

cache = refresh_cache(tickers)

# Quick preview
df_metrics = stock_df_from_cache(cache)
df_methods = method_df_from_cache(cache)

print("\n=== Metrics preview ===")
print(df_metrics.head())

print("\n=== Methods preview ===")
print(df_methods.head())

quick_df = build_quick_table(cache)
print("\n=== Quick View Table ===")
print(quick_df.head())
quick_df.to_csv("quick_view.csv", index=False)


# Optional: Save preview to CSV for quick reference
df_metrics.to_csv("metrics_preview.csv", index=False)
df_methods.to_csv("methods_preview.csv", index=False)

print("\n💾 Cache updated & preview saved.")


Loaded 1345 tickers from ETF list.
✅ No missing tickers — cache is complete.

=== Metrics preview ===
  Ticker          Short Name   Price  Pe Ratio  Peg Ratio  Eps Ttm  \
0   TSLA         Tesla, Inc.  329.65    196.22       6.07     1.68   
1   AAPL          Apple Inc.  229.35     34.75       2.00     6.60   
2   AMZN    Amazon.com, Inc.  222.69     33.95       2.64     6.56   
3  GOOGL       Alphabet Inc.  201.42     21.50       1.46     9.37   
4   NVDA  NVIDIA Corporation  182.70     59.13       1.75     3.09   

   Total Revenue  Growth  3-4 Year Sales Growth   Cagr  ...  Dividend Yield%  \
0   9.272000e+10   32.30                  81.50  18.00  ...             0.00   
1   4.086250e+11   17.39                   6.89   1.87  ...             0.00   
2   6.700380e+11   12.86                  35.79   8.86  ...             0.00   
3   3.713990e+11   14.72                  35.86   8.88  ...             0.01   
4   1.485150e+11   33.74                 384.87  55.00  ...             0.00 


18 Failed downloads:
['823.HK', '66.HK', '388.HK', '1CO', '288.HK', '669.HK', '16.HK', '6.HK', '2.HK', '83.HK', '11.HK', '1.HK', '19.HK', '2299955D.TO', '4.HK', '27.HK', '3.HK', '12.HK']: YFPricesMissingError('possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")')

37 Failed downloads:
['ABN', 'BOL', 'BMW', 'AMUN', 'BPE', 'BEI', 'BVI', 'BIM', 'BNP', 'AUD.AX', 'ACS', 'ANA', 'CABK', 'AD', 'BESI']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')
['A5G', 'ALO', 'CAD.TO', 'AENA', 'BAMI', 'ADS', 'BP..L', 'AKZA', 'AV..L', 'AGN', 'BMW3', 'AKE', 'BT.A.L', 'BAYN', 'BIRG', 'BRKB', 'BFB', 'BZFUT', 'ASRNL', 'BAS', 'BA..L', 'ADYEN']: YFPricesMissingError('possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")')

31 Failed downloads:
['ELE', 'CVC', 'DHL', 'EN', 'COV', 'DBK', 'EVD', 'CLNX', 'CPR', 'DIE', 'EDP', 'ENI', 'ELI']: YFPricesMissingError('possibly delisted; 


=== Quick View Table ===
  Ticker                Name                  Sector     P/E    P/B   PEG  \
0   TSLA         Tesla, Inc.       Consumer Cyclical  196.22  13.75  6.07   
1   AAPL          Apple Inc.              Technology   34.75  51.76  2.00   
2   AMZN    Amazon.com, Inc.       Consumer Cyclical   33.95   7.11  2.64   
3  GOOGL       Alphabet Inc.  Communication Services   21.50   6.72  1.46   
4   NVDA  NVIDIA Corporation              Technology   59.13  53.14  1.75   

    D/E  ROIC  Dividend Yield %   1M %   1Y %  Peter Lynch  Personal Model  
0  0.17  0.10              0.00  11.41  64.82         0.33            37.5  
1  1.54  0.78              0.00   8.62   6.56         1.00            75.0  
2  0.48  0.22              0.00   0.07  33.40         0.76            87.5  
3  0.11  0.36              0.01  14.04  23.66         1.37           100.0  
4  0.12  0.96              0.00  12.17  74.47         1.14            87.5  

💾 Cache updated & preview saved.
